# Deploying the model as service 

In [1]:
!pip install wandb
!pip install onnxruntime

     |████████████████████████████████| 138 kB 5.0 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

conda 4.10.3 requires ruamel_yaml_conda>=0.11.14, which is not installed.
requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.8 which is incompatible.
dvc 2.5.4 requires ruamel.yaml>=0.16.1, but you'll have ruamel-yaml 0.15.87 which is incompatible.
bentoml 0.13.1 requires urllib3<=1.25.11, but you'll have urllib3 1.26.8 which is incompatible.


In [3]:
!pip install -q bentoml --use-feature=2020-resolver "skl2onnx==1.7.0" "onnx==1.7.0" "onnxmltools==1.7.0"

In [4]:
import os
import wandb
import pandas as pd
import numpy as np
import onnxruntime as rt

## Download the model from Weights and Biases

In [5]:
import os
os.environ["WANDB_API_KEY"] = "2217d1580a9bb6478b3f7d6b4a501bb5a7f2d49d"

In [6]:
run = wandb.init(project='usedCarV2')

wandb: Currently logged in as: manaranjan (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [7]:
ARTIFACT_NAME = 'XGBoost_UserCar:v0'
#run = wandb.init()
artifact = run.use_artifact('manaranjan/usedCarV2/' + ARTIFACT_NAME, type='model')
artifact_dir = artifact.download()
artifact_dir

'./artifacts/XGBoost_UserCar:v0'

In [8]:
!ls -al  {artifact_dir}

total 584
drwxr-xr-x  3 manaranjan  staff      96 Feb 18 12:13 .
drwxr-xr-x  3 manaranjan  staff      96 Feb 18 12:13 ..
-rw-r--r--  1 manaranjan  staff  296292 Feb 18 12:13 usedcar_xgboost.onnx


## Creating the Model Service

In [9]:
%%writefile usedcar_price_prediction.py

import pandas as pd
import numpy as np
import bentoml
from bentoml.adapters import JsonInput
from bentoml.frameworks.onnx import OnnxModelArtifact

@bentoml.artifacts([OnnxModelArtifact('model')])
@bentoml.env(infer_pip_packages=True)
class UsedCarPriceOnnx(bentoml.BentoService):
    
    @bentoml.api(input=JsonInput(), batch=False)
    def predict(self, json):

      print(json)

      df = pd.DataFrame(json, index = [0])

      x_columns = ['KM_Driven', 'Fuel_Type', 'age',
                    'Transmission', 'Owner_Type', 'Seats',
                    'make', 'mileage_new', 'engine_new', 'model',
                    'power_new', 'Location']

      cat_features = ['Fuel_Type',
                      'Transmission', 'Owner_Type', 'model',
                      'make', 'Location']

      num_features = list(set(x_columns) - set(cat_features))       

      inputs = {c: df[c].values for c in df.columns}
      for c in num_features:
        inputs[c] = inputs[c].astype(np.float32)
      for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))  

      outputs = self.artifacts.model.run(None, inputs)

      return f"Expected Sale Price of your car is: INR {np.round(float(outputs[0][0,0]), 1)} Lakhs"

Writing usedcar_price_prediction.py


In [10]:
import onnx

# onnx_model is an in-memory ModelProto
onnx_model = onnx.load(artifact_dir + "/usedcar_xgboost.onnx")

In [11]:
from usedcar_price_prediction import UsedCarPriceOnnx

In [12]:
svc = UsedCarPriceOnnx()
svc.pack('model', onnx_model)

saved_path = svc.save()

[2022-02-18 12:13:14,992] INFO - BentoService bundle 'UsedCarPriceOnnx:20220218121313_B9610B' saved to: /Users/manaranjan/bentoml/repository/UsedCarPriceOnnx/20220218121313_B9610B


In [13]:
saved_path

'/Users/manaranjan/bentoml/repository/UsedCarPriceOnnx/20220218121313_B9610B'

## Serving the model as service

### Starting the service

In [14]:
!bentoml serve UsedCarPriceOnnx:latest --run-with-ngrok

[2022-02-18 12:13:19,113] INFO - Getting latest version UsedCarPriceOnnx:20220218121313_B9610B
[2022-02-18 12:13:19,130] INFO - Starting BentoML API proxy in development mode..
[2022-02-18 12:13:19,694] INFO - Starting BentoML API server in development mode..
 * Serving Flask app "UsedCarPriceOnnx" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:60088/ (Press CTRL+C to quit)
[2022-02-18 12:13:21,565] INFO - Your system nofile limit is 4864, which means each instance of microbatch service is able to hold this number of connections at same time. You can increase the number of file descriptors for the server process, or launch more microbatch instances to accept more concurrent connection.
======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)
127.0.0.1 - - [18/Feb/2022 12:13:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 12:13:26] "GET /static_content/main.css HTTP/1.1" 200 -
127.

765{'KM_Driven': 10, 'Fuel_Type': 'Petrol', 'age': 4, 'Transmission': 'Automatic', 'Owner_Type': 'First', 'Seats': 4, 'make': 'maruti', 'mileage_new': 21, 'engine_new': 900, 'model': 'swift', 'power_new': 90, 'Location': 'Bangalore'}
[2022-02-18 12:17:46,955] INFO - {'service_name': 'UsedCarPriceOnnx', 'service_version': '20220218121313_B9610B', 'api': 'predict', 'task': {'data': '{"KM_Driven":10,"Fuel_Type":"Petrol","age":4,"Transmission":"Automatic","Owner_Type":"First","Seats":4,"make":"maruti","mileage_new":21,"engine_new":900,"model":"swift","power_new":90,"Location":"Bangalore"}', 'task_id': '73967499-58b0-4c76-8f87-1fb5d431cc35', 'http_headers': (('Host', '127.0.0.1:60088'), ('User-Agent', 'python-requests/2.24.0'), ('Accept-Encoding', 'gzip, deflate'), ('Accept', '*/*'), ('Connection', 'keep-alive'), ('Content-Length', '207'))}, 'result': {'data': '"Expected Sale Price of your car is: INR 8.0 Lakhs"', 'http_status': 200, 'http_headers': (('Content-Type', 'application/json'),)},

{"KM_Driven":10.0,"Fuel_Type":"Petrol","age":4.0,"Transmission":"Automatic","Owner_Type":"First","Seats":4.0,"make":"maruti","mileage_new":21.0,"engine_new":900.0,"model":"swift","power_new":90.0,"Location":"Bangalore"}

curl -X POST "http://c19f-34-125-188-233.ngrok.io/predict" -H "accept: */*" -H "Content-Type: application/json" -d "{\"KM_Driven\":10,\"Fuel_Type\":\"Petrol\",\"age\":4,\"Transmission\":\"Automatic\",\"Owner_Type\":\"First\",\"Seats\":4,\"make\":\"maruti\",\"mileage_new\":21,\"engine_new\":900,\"model\":\"swift\",\"power_new\":90,\"Location\":\"Bangalore\"}"